In [1]:
import pandas as pd
import json
import concurrent.futures
from openai import OpenAI
import time
import numpy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import threading

In [2]:

# Initialize multiple LLM clients
clients = [
    OpenAI(base_url="http://localhost:123/v1", api_key="sk_1234567890"),
    OpenAI(base_url="http://localhost:1234/v1", api_key="sk_1234567890"),
    OpenAI(base_url="http://localhost:12345/v1", api_key="sk_1234567890")
]


In [3]:

BOOL_SYSTEM_MESSAGE = """You are excellent message moderator, expert in detecting fraudulent messages.

You will be given "Messages" and your job is to predict if a message is fraudulent or not.

You ONLY respond FOLLOWING this json schema:

{
    "is_fraudulent": {
        "type": "boolean",
        "description": "Whether the message is predicted to be fraudulent."
    }
}

You MUST ONLY RESPOND with a boolean value in JSON. Either true or false in JSON. NO EXPLANATIONS OR COMMENTS.

Example of valid responses:
{
    "is_fraudulent": true
}
or 
{
    "is_fraudulent": false
}
"""

In [4]:
# Define multiple clients for different model instances
clients = {
    "model1": OpenAI(base_url="http://localhost:123/v1", api_key="sk_1234567890"),
    "model2": OpenAI(base_url="http://localhost:1234/v1", api_key="sk_1234567890"),
    "model3": OpenAI(base_url="http://localhost:12345/v1", api_key="sk_1234567890")
}

BOOL_SYSTEM_MESSAGE = """[Your system message as before]"""

system_message = BOOL_SYSTEM_MESSAGE

def predict_fraudulence_modified(client, sms_text):
    # Similar to your original function, but now uses the client passed as an argument
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": sms_text},
            ],
            temperature=0.3,
        )
        prediction_content = response.choices[0].message.content
        try:
            prediction_json = json.loads(prediction_content)
            if "is_fraudulent" in prediction_json and type(prediction_json["is_fraudulent"]) == bool:
                return prediction_json['is_fraudulent'], 'valid'
            else:
                # Record the raw response if JSON is invalid but well-formed
                return prediction_content, 'invalid_json'
        except json.JSONDecodeError:
            # Record the raw response if JSON structure is invalid
            return prediction_content, 'invalid_json_structure'
    except Exception as e:
        return str(e), 'error'

def process_batch(client_key, df_subset):
    for index, row in df_subset.iterrows():
        sms_text = row['SMS test']  # Make sure the column name matches your dataset
        predicted_label, prediction_type = predict_fraudulence_modified(clients[client_key], sms_text)
        df_subset.at[index, 'Predicted'] = predicted_label
        df_subset.at[index, 'PredictionType'] = prediction_type
    # Save partial results for each subset
    df_subset.to_csv(f'../dataset/sms_predictions_{client_key}.csv', index=False)

# Load the dataset
try:
    df = pd.read_csv('../dataset/sms.csv')
    df['Predicted'] = None
    df['PredictionType'] = None
except FileNotFoundError:
    print("Dataset file not found.")
    exit(1)

# Splitting the dataset into three parts
df_parts = np.array_split(df, 3)

# Create and start threads for parallel processing
threads = []
for i, client_key in enumerate(clients.keys()):
    thread = threading.Thread(target=process_batch, args=(client_key, df_parts[i]))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

# Combine the parts back into a single DataFrame
df_final = pd.concat([pd.read_csv(f'../dataset/sms_predictions_{client_key}.csv') for client_key in clients.keys()])

# Calculate and print metrics
valid_predictions_df = df_final[df_final['PredictionType'] == 'valid']
accuracy = accuracy_score(valid_predictions_df['Fraudulent'], valid_predictions_df['Predicted'])
precision = precision_score(valid_predictions_df['Fraudulent'], valid_predictions_df['Predicted'])
recall = recall_score(valid_predictions_df['Fraudulent'], valid_predictions_df['Predicted'])
f1 = f1_score(valid_predictions_df['Fraudulent'], valid_predictions_df['Predicted'])

print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")


d:\Github\AI-PROJECT-LUISS\.venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# WIP